<a href="https://colab.research.google.com/github/AmanPriyanshu/Weird-Keras-Models/blob/master/PeelingAMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
import math
import tensorflow as tf
#from tqdm.notebook import tqdm
from tqdm import tqdm, trange

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
def clockwise(arr):
  borders = arr[0]
  arr = arr[1:]
  arr = arr.T[::-1]
  if arr.shape[0] == 1:
    return np.concatenate((borders, arr[0]))
  borders = np.concatenate((borders, clockwise(arr)))
  return borders

In [4]:
np.random.seed(0)
array = np.random.choice(50, (3, 4), replace=False)
print(array)

[[28 11 10 41]
 [ 2 27 38 31]
 [22  4 33 35]]


In [5]:
print(clockwise(array))

[28 11 10 41 31 35 33  4 22  2 27 38]


In [6]:
def pad(arr, n=100):
  x = np.zeros((n, n))
  for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
      x[i][j] = arr[i][j]
  return x

In [7]:
x = np.array([np.random.choice(36, (random.randint(3, 5), random.randint(3, 6)), replace=False) for _ in trange(15000)])
y = np.array([clockwise(i) for i in tqdm(x)])
y = np.array([np.concatenate((i, np.zeros(64-i.shape[0]))) for i in tqdm(y)])
x_padded = np.array([pad(i) for i in tqdm(x)])

100%|██████████| 15000/15000 [00:00<00:00, 19063.46it/s]


In [8]:
model = tf.keras.Sequential([
                             tf.keras.layers.Input(shape=(100, 100, 1)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(1024, activation='relu'),
                             tf.keras.layers.Dense(1024, activation='relu'),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(64, activation='relu'),
])

In [9]:
reduce_lr  = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto',
    min_delta=0.0001, cooldown=0, min_lr=0
)

In [10]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [11]:
class TQDMCallbacks(tf.keras.callbacks.Callback):
  def __init__(self, max_epochs):
    self.t = tqdm(total=max_epochs)
    self.max_epochs = max_epochs
  def on_epoch_end(self, epoch, logs=None):
    desc = {}
    for k,i in logs.items():
      desc.update({k:"{:.4f}".format(i)})
    self.t.update(1)
    self.t.set_postfix(desc)
    self.t.refresh()

  def on_train_end(self, logs=None):
    self.t.close()
    print("\nIt ended")

In [12]:
model.fit(x_padded, y, validation_split=0.15, epochs=500, callbacks=[reduce_lr, TQDMCallbacks(500)], verbose=0, batch_size=1000)

100%|██████████| 500/500 [03:12<00:00,  2.59it/s, loss=0.5820, mae=0.0807, val_loss=0.6116, val_mae=0.0955, lr=0.0000]


It ended


In [13]:
model.evaluate(x_padded, y)
model.summary()

469/469 [==============================] - 1s 2ms/step - loss: 0.5864 - mae: 0.0829
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              10241024  
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
______________________________________

In [14]:
output = model.predict(np.array([x_padded[0]]))

In [15]:
x[0]

array([[27,  2, 16, 32,  8, 31],
       [23, 13, 20, 19, 30, 28],
       [22, 12, 33, 21, 18, 34],
       [14, 11,  6,  3, 29,  7],
       [25,  1, 24, 26, 10,  9]])

In [16]:
output = np.array([i for i in output[0]])

In [17]:
o = []
for i in output:
  flatline = x[0].flatten()
  answers = np.argmin(np.abs(flatline - i))
  o.append(flatline[answers])
o = o[:flatline.shape[0]]

In [18]:
print(o)
print([i for i in clockwise(x[0])])

[27, 2, 16, 32, 8, 31, 28, 34, 7, 9, 10, 26, 24, 1, 25, 14, 22, 23, 13, 20, 19, 30, 18, 29, 3, 6, 11, 12, 1, 21]
[27, 2, 16, 32, 8, 31, 28, 34, 7, 9, 10, 26, 24, 1, 25, 14, 22, 23, 13, 20, 19, 30, 18, 29, 3, 6, 11, 12, 33, 21]
